In [1]:
import pandas as pd
import gmaps
import time
import gmaps.datasets
import requests
from random import randint
from datetime import datetime
from time import sleep
from csv import DictWriter, DictReader

In [2]:
GOOGLE_API_KEY='AI....'  # Fill in with your API key !

In [3]:
VK_VERSION = '5.92'
VK_ACCESS_TOKEN = 'd059....'   # Fill in with your API key !

In [4]:
# geo = (45.190445, 33.366861) # evpatoria
evpteatr = (45.190810, 33.367055) # teatr square Evpatoria
simfsovets = (44.952125, 34.102454) # soviet square Simferopol
sevasvokzal = (44.595024, 33.528891) # railway station Sevastopol
sevaskamish = (44.568270, 33.440542) # kamishovaya bay Sevastopol 
yalta = (44.498231, 34.169317) # administration Yalta
dist = 3000
maxTS = 1546300800 # 00:00 1.01.2019
minTS = 1514764800 # 00:00 1.01.2018

In [6]:
def getVK(geo, minTimestamp, maxTimestamp, offset):
    params = {
        'lat': geo[0],
        'long': geo[1],
        'count': '1000',
        'offset': offset,
        'radius': dist,
        'start_time': minTimestamp,
        'end_time': maxTimestamp,
        'access_token': VK_ACCESS_TOKEN,
        'v': VK_VERSION,
        'sort': 0 # by date of creation
    }
    return getResponse("https://api.vk.com/method/photos.search",
                       params=params, verify=True).json()

In [7]:
# resp = getVK(geo[0], geo[1], dist, minTS, maxTS, 0) # 'count': 411874, max_count = 1000

In [8]:
def save_to_csv(data, path):
    with open(path, 'w', newline='', encoding='utf-8') as csvfile:
        columns = list(data[0].keys())
        columns.sort()
        writer = DictWriter(csvfile, fieldnames=columns, delimiter=';')
        writer.writeheader()
        writer.writerows(data)

In [9]:
def save_map_to_csv(map_data, path):
    with open(path, 'w', newline='', encoding='utf-8') as csvfile:
        columns = ['id', 'date', 'lat', 'long']
        writer = DictWriter(csvfile, fieldnames=columns, delimiter=';')
        writer.writeheader()
        for key in map_data:
            data = {
                'id': key[0],
                'date': time.strftime("%d.%m.%Y", time.localtime(key[1])),
                'lat': map_data[key][0],
                'long': map_data[key][1],
            }
            writer.writerow(data)

In [10]:
# resp = getVK(geo[0], geo[1], dist, minTS, maxTS, 826)

In [11]:
# len(resp['response']['items'])

In [12]:
def open_csv_to_map(map_data, path):
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = DictReader(csvfile, delimiter=';')
#         print(reader)
        for row in reader:
            map_data[(int(row['id']), 
                      int(time.mktime(time.strptime(row['date'], '%d.%m.%Y'))))] = 
                (float(row['lat']), float(row['long']))

In [13]:
# parsing the server response into map_data[]
def save_to_map(resp):
    try:
        items = resp['response']['items']
    except KeyError:
        return
    for f in items:
        try:
            map_data[(f['id'], f['date'])] = (f['lat'], f['long'])
        except KeyError:
            continue

In [14]:
def save_to_list(resp, data, geotags):
    for f in resp['response']['items']:
        data.append({
            'lat': f['lat'], 
            'long': f['long'], 
            'date': f['date'], 
            'id': f['id']
        })
        geotags.append((f['lat'], f['long']))

In [15]:
map_data = {}
step = 24*60*60 # step - 1 day
i = minTS
geo = yalta
while i < maxTS:
    resp = getVK(geo, i, i+step, 0)
#     save_to_list(resp, data, evpphotos)
    save_to_map(resp)
    count = resp['response']['count']
    returned = len(resp['response']['items'])
    print ('{}: count {} ret {}'.format(time.strftime("%d.%m.%Y", time.localtime(i)), count, returned))
    if count > returned:
        offset = returned
        while offset < count and offset < 3000:
            resp = getVK(geo, i, i+step, offset)
#             save_to_list(resp, data, evpphotos)
            save_to_map(resp)
            count = resp['response']['count']
            returned = len(resp['response']['items'])
#             print ('{}: count {} ret {} off {}'.format(time.strftime("%d.%m.%Y", time.localtime(i)), count, returned, offset))
            offset = offset + returned
            if returned == 0:
                break
    i = i + step
    sleep(0.5)      

01.01.2018: count 1136 ret 627
02.01.2018: count 1271 ret 801
03.01.2018: count 1541 ret 485
04.01.2018: count 1149 ret 678
05.01.2018: count 1253 ret 816
06.01.2018: count 1127 ret 566
07.01.2018: count 1701 ret 738
08.01.2018: count 1144 ret 834
09.01.2018: count 1080 ret 614
10.01.2018: count 957 ret 627
11.01.2018: count 610 ret 610
12.01.2018: count 713 ret 713
13.01.2018: count 461 ret 461
14.01.2018: count 782 ret 600
15.01.2018: count 533 ret 500
16.01.2018: count 681 ret 677
17.01.2018: count 846 ret 392
18.01.2018: count 530 ret 530
19.01.2018: count 464 ret 464
20.01.2018: count 536 ret 536
21.01.2018: count 634 ret 634
22.01.2018: count 456 ret 456
23.01.2018: count 521 ret 521
24.01.2018: count 375 ret 375
25.01.2018: count 453 ret 453
26.01.2018: count 640 ret 640
27.01.2018: count 624 ret 624
28.01.2018: count 1192 ret 771
29.01.2018: count 594 ret 594
30.01.2018: count 626 ret 626
31.01.2018: count 634 ret 634
01.02.2018: count 682 ret 497
02.02.2018: count 643 ret 443


In [16]:
save_map_to_csv(map_data, 'yalta2018.csv')

In [21]:
print('Count of photos {}'.format(len(map_data)))
gmaps.configure(api_key=GOOGLE_API_KEY)
fig = gmaps.figure(center=geo, zoom_level=13)
heatmap_layer = gmaps.heatmap_layer(list(map_data.values()))
fig.add_layer(heatmap_layer)
fig

Count of photos 422546


Figure(layout=FigureLayout(height='420px'))

In [49]:
#  read from file
map_data = {}
open_csv_to_map(map_data, 'sevas2018.csv')

In [ ]:
# simple check first 5 row
k = 0 
for i in map_data:
    print(i, map_data[i])
    if k > 5: break
    k = k+1

In [11]:
gmaps.configure(api_key=GOOGLE_API_KEY)
fig = gmaps.figure(center=geo, zoom_level=13)
locations = [geo]
heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))